In [1]:
import requests
from pandas import DataFrame,read_csv, concat
from io import StringIO
import omero

from py2cytoscape.data.cyrest_client import CyRestClient
import py2cytoscape.util.dataframe as cydf
import numpy as np
from IPython.display import Image
import py2cytoscape.cytoscapejs as renderer

<IPython.core.display.Javascript object>

In [2]:
# TODO: Remove this once it's added to the default path
import sys
sys.path.append('../library')

from idr_demo import connection

conn = connection()

In [3]:
def getBulkAnnotationAsDf(screenID,conn):
#    ofId=8118685
    sc=conn.getObject('Screen',screenID)
    for ann in sc.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            if (ann.getFile().getName()=='bulk_annotations'):
                if (ann.getFile().getSize()> 147625090): #about 140Mb?
                    print "that's a big file..."
                    return None
                ofId=ann.getFile().getId()
                break


    original_file = omero.model.OriginalFileI(ofId, False)

    openTable = conn.c.sf.sharedResources().openTable(original_file)
    rowCount = openTable.getNumberOfRows()

    #table to df

    column_names = [col.name for col in openTable.getHeaders()]

    black_list = []
    column_indices = []
    for column_name in column_names:
        if column_name in black_list:
            continue
        column_indices.append(column_names.index(column_name))

    table_data = openTable.slice(column_indices, None)
    data = []
    for index in range(rowCount):
        row_values = [column.values[index] for column in table_data.columns]
        data.append(row_values)

    dfAnn=DataFrame(data)
    dfAnn.columns=column_names
    return dfAnn


In [4]:
def getGenesFromPhenotype(df,phTerm):
    colElong=[]
    for col in df.columns:
        if ('Term Accession' in col):
            if (phTerm in df[col].unique()):
                colElong.append(col)

    dfElong=concat([df[df[col]!= ''] for col in colElong])
    return dfElong['Gene Identifier'].unique()
    

## Querying the IDR

In [5]:
#CMPO term to look for
phTerm='CMPO_0000077'

#ids of screens:
#scId=3 #Graml et al.
#scId=206 #Rohn et al., B
#scId=1202 #Fuchs et al., B

screens=[3,206,1202]

genes=[]
for scId in screens:
    print 'loading '+str(scId)

    #loading bulk_annotations of screens as dataframes
    df=getBulkAnnotationAsDf(scId,conn)
    
    #unique genes with CPMO term
    cur=getGenesFromPhenotype(df,phTerm)
    print 'got '+str(len(cur))+ ' genes'
    genes.extend(cur)

loading 3
got 39 genes
loading 206
got 72 genes
loading 1202
got 90 genes


In [6]:
#translation; the table was build off line using biomart
dfTrans=read_csv('TableOfGenesWithElongatedCellPhenotype.csv')

#extract IDs 
genesE84=concat([dfTrans[dfTrans['Screen GeneID']==x]['Human Ortholog Ensembl 84'] for x in genes])
genesE84=genesE84[genesE84!='(null)']

print genes[:10]
print genesE84.head(10)

['SPAC3G9.08', 'SPCC970.10c', 'SPCC553.08c', 'SPAC16C9.05', 'SPAC30.02c', 'SPAC3H8.05c', 'SPCC306.04c', 'SPCC338.16', 'SPCP1E11.06', 'SPAC1556.01c']
15    ENSG00000071243
16    ENSG00000111653
17    ENSG00000168395
18    ENSG00000168556
47    ENSG00000103549
48    ENSG00000155827
44    ENSG00000140598
2     ENSG00000109118
11    ENSG00000198841
40    ENSG00000099381
Name: Human Ortholog Ensembl 84, dtype: object


In [7]:
genesIn=[]
genesNotIn=[]
for g in genes:
    cur=dfTrans[dfTrans['Screen GeneID']==g]['Human Ortholog Ensembl 84']
    if ((len(cur)>0) and (( cur != '(null)').any())):
        genesIn.append(g)
    else:
        genesNotIn.append(g)
with open('genesIn.txt', 'w') as thefile:
    for item in genesIn:
        print>>thefile, item
with open('genesNotIn.txt', 'w') as thefile:
    for item in genesNotIn:
        print>>thefile, item

print len(genesNotIn)

18


## REST query of STRING db

In [8]:
#building STRINdb REST api query
url='http://string-db.org/api/psi-mi-tab/interactionsList?identifiers='
for g in genesE84:
    url=url+g+'%250D'
Res = requests.get(url)

In [9]:
df=read_csv(StringIO(Res.text),sep='\t',header=None)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,string:9606.ENSP00000301788,string:9606.ENSP00000215587,POLR2G,POLR2E,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.175|escore:0.999|dscore:0...
1,string:9606.ENSP00000340896,string:9606.ENSP00000264515,ASH2L,RBBP5,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.065|escore:0.999|dscore:0...
2,string:9606.ENSP00000342889,string:9606.ENSP00000215587,POLR2K,POLR2E,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.184|escore:0.999|dscore:0...
3,string:9606.ENSP00000342889,string:9606.ENSP00000301788,POLR2K,POLR2G,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.094|escore:0.999|dscore:0...
4,string:9606.ENSP00000324124,string:9606.ENSP00000215587,POLR2L,POLR2E,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.491|escore:0.999|dscore:0...
5,string:9606.ENSP00000390475,string:9606.ENSP00000262519,CXXC1,SETD1A,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.14|escore:0.999|dscore:0....
6,string:9606.ENSP00000348461,string:9606.ENSP00000269321,RAC1,ARHGDIA,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.096|escore:0.999|dscore:0...
7,string:9606.ENSP00000472929,string:9606.ENSP00000400175,VAV1,RHOA,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|escore:0.812|dscore:0.9|tscore:0.974
8,string:9606.ENSP00000314458,string:9606.ENSP00000269321,CDC42,ARHGDIA,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.071|escore:0.98|dscore:0....
9,string:9606.ENSP00000362948,string:9606.ENSP00000265350,MED18,MED20,-,-,-,-,-,taxid:9606,taxid:9606,-,-,-,score:0.999|ascore:0.072|escore:0.999|dscore:0...


## Building Cytoscape network

In [10]:

### Using the cytoscape REST api, needs a cytoscape desktop app running 


# Create Client
cy = CyRestClient()
# Clear current session
cy.session.delete()

#create network (will appear in the cytoscape desktop windows)
net1 = cy.network.create_from_dataframe(df, source_col=2,target_col=3,interaction_col=10)

cy.layout.apply(network=net1)
cy.layout.fit(network=net1)

style_s3 = cy.style.create('Sample3')
cy.style.apply(style=style_s3, network=net1)

#get display as static render of cytoscape
Image(net1.get_png(height=400))

#dynamic rendering using cytoscape.js

style_for_widget = cy.style.get(style_s3.get_name(), data_format='cytoscapejs')
renderer.render(net1.get_first_view(), style=style_for_widget['style'], background='#000000')

ConnectionError: HTTPConnectionPool(host='localhost', port=1234): Max retries exceeded with url: /v1/styles/visualproperties (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7fae776d2ad0>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
#direct dynamic rendering using cytoscape.js

renderer.render(cydf.from_dataframe(df, source_col=2,target_col=3,interaction_col=10),layout_algorithm='circle',style='Big Labels')